<a href="https://colab.research.google.com/github/sudhir2016/India_climate_map/blob/master/India_climate_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Geopandas,Rasterio and rioxarray.

In [ ]:
!pip install geopandas

In [ ]:
!pip install rasterio

In [ ]:
!pip install rioxarray

Import various libraries.

In [4]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import fiona as fn
import rasterio as rio
import xarray as xa
import rioxarray as rioxa
from rasterio import mask
from sklearn.cluster import KMeans

Mount Google Drive as the climate data is saved there.

In [5]:
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

Load climate data.

In [7]:
data='/content/drive/My Drive/adaptor.mars.internal-1599570962.375764-24582-1-fe6db266-a043-47ab-b3c2-5f6f4072408a.nc'

Open dataset using Xarray.

In [8]:
x=xa.open_dataset(data)

Set crs as epsg 4326.

In [ ]:
x.rio.set_crs('epsg:4326')

Create and open tif files for precipitation and temp.

In [10]:
x['tp'].rio.to_raster('tp.tif')

In [11]:
x['t2m'].rio.to_raster('t2m.tif')

In [12]:
tp=rio.open('tp.tif')

In [13]:
t2m=rio.open('t2m.tif')

Read India Geojason file and use it to mask the precipitation and temp data.

In [14]:
india='/content/India.json'

In [15]:
map=gpd.read_file(india)

In [ ]:
map.plot()

In [17]:
with fn.open(india,'r') as shapefile:
  shapes=[feature['geometry'] for feature in shapefile]

In [18]:
tp,tp_transform=rio.mask.mask(tp,shapes,crop=True)

In [19]:
t2m,t2m_transform=rio.mask.mask(t2m,shapes,crop=True)

Extract precipitation and temp data as array and reshape it to suit KMeans.

In [ ]:
l=tp.shape[0]
h=tp.shape[1]
w=tp.shape[2]
print(l,h,w)

In [21]:
c=np.zeros((h,w,2*l))

In [22]:
for j in range (h):
  for k in range (w):
    c[j,k,:l]=tp[:,j,k]
    c[j,k,l:]=t2m[:,j,k]

In [23]:
img=c.reshape((c.shape[0]*c.shape[1],c.shape[2]))

In [ ]:
img.shape

Run KMeans with number of clusters as 5.

In [25]:
k=KMeans(n_clusters=5)

In [ ]:
lab=k.fit_predict(img)
print(lab)

In [ ]:
k.cluster_centers_

Reshape labels and visualize.

In [28]:
lab=lab.reshape(( c.shape[0],c.shape[1]))

In [29]:
cmap = mpl.colors.ListedColormap(['w', 'g','#D3D3D3','b','c'])

In [ ]:
plt.imshow(lab,cmap=cmap)